<a href="https://colab.research.google.com/github/haosulab/SAPIEN-tutorial/blob/master/robotics/2_drive_robot_with_pid_controller.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> Note: Some core features of SAPIEN are not available on Colab, including the interactive viewer and ray-tracing functionalities. You need to run SAPIEN locally for full features. You can also find the latest SAPIEN tutorial at [SAPIEN's documentation](https://sapien.ucsd.edu/docs/latest/index.html).

# Robotics Tutorial 2: Drive Robot with PID Controller

A fundamental problem in robotics is how to apply forces on the joints of a robot to drive them to target positions. Such low-level control is the basis for applications, e.g., following a trajectory.

In this tutorial, you will learn the following:

- Drive the robot with the PhysX internal PD controller
- Write your own PID controller

## Preparation

> Note: you need GPU runtime to run the notebook.

In [ ]:
%pip install sapien

import sapien.core as sapien
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image

try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

%matplotlib inline

## Setup

As before, we first set up the simulation world. Note that we decrease the timestep, which is helpful for the simple PID controller implemented in this example.

In [ ]:
engine = sapien.Engine()
engine.set_log_level("critical")
renderer = sapien.SapienRenderer()
engine.set_renderer(renderer)

scene_config = sapien.SceneConfig()
scene = engine.create_scene(scene_config)
# A small timestep for higher control accuracy
scene.set_timestep(1 / 2000.0)
scene.add_ground(0)

scene.set_ambient_light([0.5, 0.5, 0.5])
scene.add_directional_light([0, 1, -1], [0.5, 0.5, 0.5])
camera = scene.add_camera(name='camera', width=1024, height=768, fovy=1.57, near=0.1, far=100)
camera.set_pose(sapien.Pose([-1.3, 0, 0.8], [0.988771, 0, 0.149438, 0]))

# Load URDF
if IN_COLAB:
    !gdown -q 10-ASSF6hiMLoMfajrTBMNn_U3YyP6-sl
    !unzip -o -q jaco2.zip
    assets_dir = "."
else:
    assets_dir = "../assets"
loader = scene.create_urdf_loader()
robot = loader.load(os.path.join(assets_dir, "jaco2/jaco2.urdf"))
robot.set_root_pose(sapien.Pose([0, 0, 0], [1, 0, 0, 0]))

# Set joint positions
arm_zero_qpos = [0, 3.14, 0, 3.14, 0, 3.14, 0]
gripper_init_qpos = [0, 0, 0, 0, 0, 0]
zero_qpos = arm_zero_qpos + gripper_init_qpos
robot.set_qpos(zero_qpos)
arm_target_qpos = [4.71, 2.84, 0.0, 0.75, 4.62, 4.48, 4.88]
target_qpos = arm_target_qpos + gripper_init_qpos

## Drive the robot with the PhysX internal PD controller

SAPIEN provides builtin PhysX **drives** (controllers) to control either the position or speed of a joint. For each active joint (with non-zero degree of freedom), we can call `set_drive_property(...)` to set its drive properties: `stiffness` and `damping`.

In [ ]:
active_joints = robot.get_active_joints()
for joint_idx, joint in enumerate(active_joints):
    joint.set_drive_property(stiffness=20, damping=5)
    joint.set_drive_target(target_qpos[joint_idx])
# Or you can directly set joint targets for an articulation
# robot.set_drive_target(target_qpos)

The drive is a **proportional derivative drive**, which applies a force as follows:

***force = stiffness * (targetPosition - position) + damping * (targetVelocity - velocity)***

The `stiffness` and `damping` can be regarded as the **P** and **D** term in a typical [PID controller](https://en.wikipedia.org/wiki/PID_controller). They implies the extent to which the drive attempts to achieve the target position and velocity respectively.

> Note: The parameters (`stiffness` and `damping`) for the internal drive in this example can not be directly used for downstream tasks like manipulation.
>
> The PhysX backend in fact integrates the drive into the PhysX solver. The force applied will be computed implicitly every simulation step.

The initial target position and velocity of a joint are zero by default. You can call `joint.set_drive_target(...)` to set the target position of a joint, or `robot.set_drive_target(...)` to set the target positions of all the joints of the robot. Similarly, you can also call `set_drive_velocity_target(...)` to set the target velocity.

> Note: If you do not balance the passive force, e.g. gravity, the robot can never reach the desired pose (but maybe a close pose) given in `set_drive_target` due to steady-state-error.

## Write Your own PID controller

You can write your own PID controller, if you need an integrator term I to compensate some steady-state-error which can not be compensated by `compensate_passive_force`. Here we provide a very simple implementation:

In [ ]:
class SimplePID:
    def __init__(self, kp=0.0, ki=0.0, kd=0.0):
        self.p = kp
        self.i = ki
        self.d = kd

        self._cp = 0
        self._ci = 0
        self._cd = 0

        self._last_error = 0

    def compute(self, current_error, dt):
        self._cp = current_error
        self._ci += current_error * dt
        self._cd = (current_error - self._last_error) / dt
        self._last_error = current_error
        signal = (self.p * self._cp) + \
            (self.i * self._ci) + (self.d * self._cd)
        return signal

In [ ]:
active_joints = robot.get_active_joints()
pids = []
pid_parameters = [
    (40, 5, 2), (40, 5, 2), (40, 5, 2), (20, 5.0, 2),
    (5, 0.8, 2), (5, 0.8, 2), (5, 0.8, 0.4),
    (0.1, 0, 0.02), (0.1, 0, 0.02), (0.1, 0, 0.02),
    (0.1, 0, 0.02), (0.1, 0, 0.02), (0.1, 0, 0.02),
]
for i, joint in enumerate(active_joints):
    pids.append(SimplePID(*pid_parameters[i]))

def pid_forward(
    pids: list, target_pos: np.ndarray, current_pos: np.ndarray, dt: float
) -> np.ndarray:
    errors = target_pos - current_pos
    qf = [pid.compute(error, dt) for pid, error in zip(pids, errors)]
    return np.array(qf)

This is only an example implementation, and the parameters are not carefully tuned. You can try to add extra tricks for integration or error propagation, to improve the stability of your own controller. The way of adding PID control during the simulation will be demonstrated in the next section.

> Note: In most cases, it is recommended to use the internal drive rather than your own PID. The PhysX internal drive is much more efficient and stable when the parameters are not carefully tuned.

## Visualization

Let's see what will happen if we run the simulation loop. Run the following cell and you will see 10 screenshots of the robot during the simulation. If you are running locally, an additional viewer window will be shown to display the animation of the simulation. You can change the `use_external_pid` variable and rerun the notebook to turn on/turn off our simple PID controller defined in previous section. If you want to disable internal controller to see how well the simple PID controller works independently, you can skip the cell where we enable internal drive when rerunning.

In [ ]:
use_external_pid = False

In [ ]:
plt.figure(figsize=(15, 6))
print("robot target qpos: ", target_qpos)

if IN_COLAB:
    for steps in range(2000):
        qf = robot.compute_passive_force(gravity=True, coriolis_and_centrifugal=True)
        
        if use_external_pid: # Enable simple PID controller
            pid_qf = pid_forward(pids, target_qpos, robot.get_qpos(), scene.get_timestep())
            qf += pid_qf
        
        robot.set_qf(qf)

        scene.step()

        if steps % 200 == 0:
            scene.update_render()
            camera.take_picture()
            rgba = camera.get_color_rgba()
            plt.subplot(2, 5, steps//200+1)
            plt.imshow(rgba[..., :3])
    
    print("robot qpos at steps 1999: ", robot.get_qpos())
else:
    from sapien.utils import Viewer
    viewer = Viewer(renderer)
    viewer.set_scene(scene)
    viewer.set_camera_xyz(x=-1.2, y=0, z=0.8)
    viewer.set_camera_rpy(r=0, p=-0.3, y=0)

    steps = 0
    while not viewer.closed:
        qf = robot.compute_passive_force(gravity=True, coriolis_and_centrifugal=True)
        
        if use_external_pid: # Enable simple PID controller
            pid_qf = pid_forward(pids, target_qpos, robot.get_qpos(), scene.get_timestep())
            qf += pid_qf
        
        robot.set_qf(qf)

        scene.step()

        if steps % 8 == 0: # Render every 8 steps
            scene.update_render()
            viewer.render()

        if steps < 2000 and steps % 200 == 0:
            scene.update_render()
            camera.take_picture()
            rgba = camera.get_color_rgba()
            plt.subplot(2, 5, steps//200+1)
            plt.imshow(rgba[..., :3])
        
        if steps == 1999:
            print("robot qpos at steps 1999: ", robot.get_qpos())

        steps += 1